# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

file_path = os.path.join('output_data','cities.csv')
cities_df = pd.read_csv(file_path)
cities_df.head(20)

,City,Latitude,Longitude,Temperature,Max Temp,Humidity,Wind Speed (mph),Cloudiness
0,buala,-8.1450,159.5921,81.45,81.45,79,3.13,99
1,hilo,19.7297,-155.0900,70.48,78.80,65,3.44,90
2,tuatapere,-46.1333,167.6833,55.00,55.00,93,15.01,97
3,okhotsk,59.3833,143.3000,39.04,39.04,72,2.64,55
4,busselton,-33.6500,115.3333,62.01,62.01,95,1.99,100
5,la ronge,55.1001,-105.2842,28.40,28.40,69,3.44,59
6,jamestown,42.0970,-79.2353,60.03,60.80,94,6.91,90
7,lebu,-37.6167,-73.6500,53.69,53.69,89,3.18,74
8,ouesso,1.6136,16.0517,72.72,72.72,94,1.07,100
9,butaritari,3.0707,172.7902,82.94,82.94,72,9.13,95


In [3]:
cities_df.loc[cities_df['Humidity']>=50,:]

,City,Latitude,Longitude,Temperature,Max Temp,Humidity,Wind Speed (mph),Cloudiness
0,buala,-8.1450,159.5921,81.45,81.45,79,3.13,99
1,hilo,19.7297,-155.0900,70.48,78.80,65,3.44,90
2,tuatapere,-46.1333,167.6833,55.00,55.00,93,15.01,97
3,okhotsk,59.3833,143.3000,39.04,39.04,72,2.64,55
4,busselton,-33.6500,115.3333,62.01,62.01,95,1.99,100
...,...,...,...,...,...,...,...,...
552,sitka,57.0531,-135.3300,46.40,46.40,87,5.75,90
553,camana,-16.6228,-72.7111,65.61,65.61,88,7.99,21
554,carauari,-4.8828,-66.8958,72.03,72.03,99,2.98,100
555,saint joseph,39.7686,-94.8466,58.50,60.01,88,8.05,90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)


In [5]:
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df['Humidity'].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
max_temp = 80
min_temp = 50
wind_speed = 10

cities_max_temp = cities_df.loc[cities_df['Temperature'] <= max_temp,:]
cities_max_temp
cities_min_temp = cities_max_temp.loc[cities_max_temp['Temperature']>min_temp,:]
cities_min_temp
cities_wind = cities_min_temp.loc[cities_min_temp['Wind Speed (mph)']<wind_speed,:]
cities_wind
cities_map = cities_wind.loc[cities_wind['Cloudiness']==0,:].reset_index()
print(len(cities_map))
cities_map

23


,index,City,Latitude,Longitude,Temperature,Max Temp,Humidity,Wind Speed (mph),Cloudiness
0,21,mar del plata,-38.0023,-57.5575,57.67,63.00,94,8.05,0
1,34,saint pete beach,27.7253,-82.7412,73.08,75.00,1,1.99,0
2,108,vallenar,-28.5708,-70.7581,59.16,59.16,71,1.19,0
3,147,fram,-26.9833,-55.8833,57.20,57.20,72,9.22,0
4,217,linqiong,30.4159,103.4609,78.84,81.00,50,2.24,0
5,219,linxia,35.6003,103.2064,69.08,69.08,16,4.54,0
6,226,reconquista,-29.1500,-59.6500,53.94,53.94,78,2.71,0
7,227,toktogul,41.8708,72.9402,69.30,69.30,38,1.19,0
8,238,comodoro rivadavia,-45.8667,-67.5000,57.20,57.20,58,4.61,0
9,247,mustafakemalpasa,40.0353,28.4117,61.81,66.20,82,5.75,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:

locations = cities_map[["Latitude", "Longitude"]]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    #"location": locations,  # philadelphia coords
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
    'fields': 'formatted_address'
}
# use iterrows to iterate through pandas dataframe
for index, row in cities_map.iterrows():

    # get location
    city_name = row['City']
    lat = row['Latitude']
    lng = row['Longitude']
    params['location'] = f'{lat},{lng}'


    # add keyword to params dict
    #params['keyword'] = city_name

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city_name} Hotel is {results[0]['name']}.")
        
        cities_map.loc[index,'Hotel Name'] =results[0]['name']
        #cities_map.loc[index, 'name'] = results[0]['name']
        cities_map.loc[index, 'Country'] = results[0]['vicinity']
        #cities_map.loc[index, 'price_level'] = results[0]['price_level']
        #cities_map.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: mar del plata.
Closest mar del plata Hotel is Mari Alt Costuras.
------------
Retrieving Results for Index 1: saint pete beach.
Closest saint pete beach Hotel is Laura A. Halprin, P.A..
------------
Retrieving Results for Index 2: vallenar.
Closest vallenar Hotel is Juan Manuel Barrera Marin.
------------
Retrieving Results for Index 3: fram.
Closest fram Hotel is CEMENTERIO LA PAZ, ITAPUA.
------------
Retrieving Results for Index 4: linqiong.
Closest linqiong Hotel is Qionglai Elite Vocational Skills Training School.
------------
Retrieving Results for Index 5: linxia.
Closest linxia Hotel is 天桥旅行社.
------------
Retrieving Results for Index 6: reconquista.
Closest reconquista Hotel is Coquetas Modas.
------------
Retrieving Results for Index 7: toktogul.
Closest toktogul Hotel is Tsentral'nyy Rynok.
------------
Retrieving Results for Index 8: comodoro rivadavia.
Closest comodoro rivadavia Hotel is Escuela Nivel Inicial Monigote.
------------
Retrievin

In [8]:
results[0]['vicinity']


'إبن رشد، قسم سفاجا، البحر الأحمر،'

In [9]:
hotel_df = cities_map.drop(['index'], axis = 1).rename(columns={'Latitude': 'Lat','Longitude':'Lng'})
#hotel_info = hotel_info.rename(columns={'Latitude': 'Lat','Longitude':'Lng'})
hotel_df

,City,Lat,Lng,Temperature,Max Temp,Humidity,Wind Speed (mph),Cloudiness,Hotel Name,Country
0,mar del plata,-38.0023,-57.5575,57.67,63.00,94,8.05,0,Mari Alt Costuras,Mar del Plata
1,saint pete beach,27.7253,-82.7412,73.08,75.00,1,1.99,0,"Laura A. Halprin, P.A.","Gulf Boulevard, St. Pete Beach"
2,vallenar,-28.5708,-70.7581,59.16,59.16,71,1.19,0,Juan Manuel Barrera Marin,"Gabriela Mistral 1355, Vallenar"
3,fram,-26.9833,-55.8833,57.20,57.20,72,9.22,0,"CEMENTERIO LA PAZ, ITAPUA",Unnamed Road
4,linqiong,30.4159,103.4609,78.84,81.00,50,2.24,0,Qionglai Elite Vocational Skills Training School,"65 Xue Dao Jie, Qionglai Shi, Chengdu Shi"
5,linxia,35.6003,103.2064,69.08,69.08,16,4.54,0,天桥旅行社,"82 Hong Yuan Lu, Linxia Shi, Linxia Huizuzizhi..."
6,reconquista,-29.1500,-59.6500,53.94,53.94,78,2.71,0,Coquetas Modas,Reconquista
7,toktogul,41.8708,72.9402,69.30,69.30,38,1.19,0,Tsentral'nyy Rynok,Toktogul
8,comodoro rivadavia,-45.8667,-67.5000,57.20,57.20,58,4.61,0,Escuela Nivel Inicial Monigote,"Maipú 1666, Comodoro Rivadavia"
9,mustafakemalpasa,40.0353,28.4117,61.81,66.20,82,5.75,0,Elbir Gross Market,Fevzi Paşa Cad. No:221


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map


# Display figure
